In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
# PAQUETES
# -----------------------------------------------------------------------------

# Base
import pandas as pd
import numpy as np
import random

# Visualización
import plotly.express as ex

# Web Scraping
import requests
from bs4 import BeautifulSoup

# trabajo con variables abocadas al tiempo
import sys
import time
from datetime import datetime
from termcolor import colored

# GC
import gc

# Itertools
import itertools

# Para extraer datos de un gráfico
from itertools import repeat    
from pprint import pprint as pp

# Expresiones regulares
import re 

# Configuración
import warnings
warnings.filterwarnings("ignore")
warnings.simplefilter(action='ignore', category=FutureWarning)
pd.set_option('display.max_columns', None)

#base de datos
# import mysql.connector
# from mysql.connector import Error
from pandas import DataFrame

from PIL import Image
import os
from pathlib import Path

# -----------------------------------------------------------------------------

def get_time(cond):
    if cond == "start":
        p = "Este proceso comenzó en "
    elif cond == "end":
        p = "Este proceso finalizó en "
    print("") 
    print(colored(p + str(datetime.now().strftime("%H:%M:%S")), "green", "on_white", attrs=["bold",'reverse', 'blink']))
    
    
def Filter(string, substr): 
        return [str for str in string if
                any(sub in str for sub in substr)] 
    
def NOTFilter(string, substr): 
    return [str for str in string if
            any(sub not in str for sub in substr)] 

In [3]:
from threading import Thread
from random import choice
from multiprocessing import Pool
from tqdm import tqdm
from queue import Queue

HEADERS =[
    {'User-Agent': 'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/47.0.2526.106 Safari/537.36'},
    {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/103.0.0.0 Safari/537.36'},
    {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.361'},
    {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:53.0) Gecko/20100101 Firefox/53.01'},
    {'User-Agent': 'Mozilla/5.0 (compatible; MSIE 9.0; Windows NT 6.0; Trident/5.0; Trident/5.0)1'},
    {'User-Agent': 'Mozilla/5.0 (compatible; MSIE 10.0; Windows NT 6.2; Trident/6.0; MDDCJS)1'},
    {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/51.0.2704.79 Safari/537.36 Edge/14.143931'},
    {'User-Agent': 'Mozilla/4.0 (compatible; MSIE 6.0; Windows NT 5.1; SV1) 1'},
    {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/104.0.0.0 Safari/537.36'}
    ]

def create_sesion(headers = {}):
    session = requests.Session()

    # create random header
    session.headers = choice(HEADERS)
    session.headers.update(headers)

    # set-up proxy
    # proxy = choice(proxiesUsable)
    # proxy = proxiesUsable
    # session.proxies = {'https':proxy, 'http':proxy}
    return session



def make_request_from_session(session, url):
    res = session.get(url)
    return BeautifulSoup(res.content.decode('utf-8', 'ignore'), "html5lib")



def make_request(url, headers = None):
    headers = headers or choice(HEADERS)
    res = requests.get(url, headers = headers )
    return BeautifulSoup(res.content.decode('utf-8', 'ignore'), "html5lib")



class Worker(Thread):
    def __init__(self, func, queue, results, pbar, session = None):
        Thread.__init__(self)
        self.queue = queue
        self._session = session or create_sesion()
        self._func = func
        self._pbar = pbar
        self._results = results


    def run(self):
        while True:
            
            # get url from Queue
            urlInfo = self.queue.get() # dictionary {'url':'','headers':''}
            
            # get url
            url =  urlInfo.get('url')

            self._session.headers.update(urlInfo.get('headers', {}))
            
            # Soup
            soup = make_request_from_session(self._session, url)
                
            # result
            _result = self._func(soup, url = url, id_ = get_id_from_url(url),
                                 refere = url)
            
            # Result queue
            self._results.put(_result)

            self._pbar.update()
            
            self.queue.task_done()
            
            # Random wait u=2, s=1 between requests
            time.sleep(np.abs(np.random.normal(loc=2, scale=1.0, size=None)))

# ========================================================================== #
# CONCURRENT FUTURES
# ========================================================================== #
_cores = os.cpu_count() # mp.cpu_count()

def setup_cores(func, urls):
    with Pool(_cores) as p:
        return p.map(func, urls)
        # return tqdm(p.imap_unordered(func, urls), total=len(urls))


# def setup_threads(func, urls):
#     # workers = x if (x:= len(urls)) < 100 else 100
#     with ThreadPoolExecutor() as executor:
#         return executor.map(func, urls) # executor._max_workers (default: os.cpu_count() + 4)


def setup_threads(func, urls, nthread=None):

    # create Queue
    queue = Queue()

    # output vector
    results = Queue()

    # Calculate number of threads
    # nthread = m if (m := len(urls)) < (n := os.cpu_count() + 4) else n
    nthread = min(*list(filter(None, [nthread, len(urls), os.cpu_count() + 4])))

    # create progress bar
    pbar = tqdm(desc = f"{func.__name__}", total = len(urls))

    # create jobs
    jobs = [Worker(func, queue = queue, results = results, pbar=pbar)  for _ in range(nthread)] # todo: meter pbar dentro del worker

    # Start workers
    [worker.start() for worker in jobs]

    #% rellenar la cola
    [queue.put(url) for url in urls]

        
    # wait until join
    queue.join()

    # Progress Bar
    pbar.close()

    # [j._result for j in jobs],
    return [results.get() for _ in range(results.qsize())] #  [results.get() for _ in range(len(urls))]

<hr>

<hr>


<h1 style="color:green" >Capturando las urls de las ligas</h1>

Usaremos esta funcion fbref_league_history para capturar las URLs historicas de las ligas.
Lo que necesitaremos antes de todas las temporadas historicas es tener las url de las competiciones que queremos obtener la info.

In [4]:
history=[
        # 'https://fbref.com/en/comps/12/La-Liga-Stats'
        # ,'https://fbref.com/en/comps/9/Premier-League-Stats'
        # , 'https://fbref.com/en/comps/11/Serie-A-Seasons'
        # , 'https://fbref.com/en/comps/13/Ligue-1-Seasons'
        # , 'https://fbref.com/en/comps/20/Bundesliga-Seasons'
        # , 'https://fbref.com/en/comps/21/Primera-Division-Seasons'
         'https://fbref.com/en/comps/24/Serie-A-Seasonss'
        , 'https://fbref.com/en/comps/22/Major-League-Soccer-Seasons'
        , 'https://fbref.com/en/comps/31/Liga-MX-Seasons'
        ]

seasons = ['2022-2023']

name_competition_season = 'four_2022_'

<hr>


<h1 style="color:green" >Capturando las urls de los Equipos</h1>

A partir de las URLs de las ligas obtenidas vamos a ir en busca de las URLs de cada equipo de esas ligas usando la función fbref_team_url_history.

In [5]:
def fbref_team_url_history(league_history):
    team_season_url = []
    for league_season_url in league_history:

        # r=requests.get(league_season_url)
        # soup=BeautifulSoup(r.content, "html.parser")
        soup=make_request(league_season_url)

        teams = soup.find("table").find_all("a")
        teams = list(map(lambda x: "https://fbref.com" + x["href"], teams))
        teams = Filter(teams, ["/en/squads/"])
        team_season_url.append(teams)

    # Todas las historias en una matriz
    team_season_url  = list(itertools.chain(*team_season_url))
    return team_season_url

# Test para la LaLiga la temporada pasada
team_season_url = fbref_team_url_history(history[:])
team_season_url

['https://fbref.com/en/squads/abdce579/Palmeiras-Stats',
 'https://fbref.com/en/squads/6f7e1f03/Internacional-Stats',
 'https://fbref.com/en/squads/84d9701c/Fluminense-Stats',
 'https://fbref.com/en/squads/bf4acd28/Corinthians-Stats',
 'https://fbref.com/en/squads/639950ae/Flamengo-Stats',
 'https://fbref.com/en/squads/2091c619/Atletico-Paranaense-Stats',
 'https://fbref.com/en/squads/422bb734/2022/Atletico-Mineiro-Stats',
 'https://fbref.com/en/squads/a9d0ab0e/2022/Fortaleza-Stats',
 'https://fbref.com/en/squads/5f232eb1/Sao-Paulo-Stats',
 'https://fbref.com/en/squads/1f68d780/America-MG-Stats',
 'https://fbref.com/en/squads/d9fdd9d9/Botafogo-RJ-Stats',
 'https://fbref.com/en/squads/712c528f/Santos-Stats',
 'https://fbref.com/en/squads/78c617cc/Goias-Stats',
 'https://fbref.com/en/squads/f98930d1/Bragantino-Stats',
 'https://fbref.com/en/squads/d680d257/Coritiba-Stats',
 'https://fbref.com/en/squads/f0e6fb14/Cuiaba-Stats',
 'https://fbref.com/en/squads/2f335e17/Ceara-Stats',
 'https:/

In [6]:
def fbref_player_url(team_season_url):

    player_url = []
    for turl in team_season_url:
        print(turl)
        try:
          # r=requests.get(turl)
          # soup=BeautifulSoup(r.content, "html.parser")
          soup=make_request(turl)

          soup.find("div", {"id":"all_stats_standard"})
          players = soup.find("tbody").find_all("a")
          players = list(map(lambda x: x["href"], players))
          players = Filter(players, ["/en/players/"])
          players = NOTFilter(players, ["matchlogs"])
          player_url.append(list(map(lambda x: "https://fbref.com" + x, players)))
          time.sleep(0.01)
        except:
          pass
    player_url  = list(itertools.chain(*player_url))
    return player_url


# Test para jugadores del sevilla. se encuentra en la 4ta posision
#player_url = fbref_player_url(["https://fbref.com/en/squads/e9ae80b7/2016/Sarmiento-Stats"])
player_url = fbref_player_url(team_season_url[:])
player_url

https://fbref.com/en/squads/abdce579/Palmeiras-Stats
https://fbref.com/en/squads/6f7e1f03/Internacional-Stats
https://fbref.com/en/squads/84d9701c/Fluminense-Stats
https://fbref.com/en/squads/bf4acd28/Corinthians-Stats
https://fbref.com/en/squads/639950ae/Flamengo-Stats
https://fbref.com/en/squads/2091c619/Atletico-Paranaense-Stats
https://fbref.com/en/squads/422bb734/2022/Atletico-Mineiro-Stats
https://fbref.com/en/squads/a9d0ab0e/2022/Fortaleza-Stats
https://fbref.com/en/squads/5f232eb1/Sao-Paulo-Stats
https://fbref.com/en/squads/1f68d780/America-MG-Stats
https://fbref.com/en/squads/d9fdd9d9/Botafogo-RJ-Stats
https://fbref.com/en/squads/712c528f/Santos-Stats
https://fbref.com/en/squads/78c617cc/Goias-Stats
https://fbref.com/en/squads/f98930d1/Bragantino-Stats
https://fbref.com/en/squads/d680d257/Coritiba-Stats
https://fbref.com/en/squads/f0e6fb14/Cuiaba-Stats
https://fbref.com/en/squads/2f335e17/Ceara-Stats
https://fbref.com/en/squads/32d508ca/Atletico-Goianiense-Stats
https://fbref.

['https://fbref.com/en/players/895ef5ec/Dudu',
 'https://fbref.com/en/players/81be82e9/Weverton',
 'https://fbref.com/en/players/a816dbfb/Danilo',
 'https://fbref.com/en/players/10fe831e/Murilo-Cerqueira',
 'https://fbref.com/en/players/2964fd20/Gustavo-Scarpa',
 'https://fbref.com/en/players/634b1413/Ze-Rafael',
 'https://fbref.com/en/players/d5ce52a5/Gustavo-Gomez',
 'https://fbref.com/en/players/bffd5e50/Rony',
 'https://fbref.com/en/players/fdd2f275/Marcos-Rocha',
 'https://fbref.com/en/players/46009f44/Joaquin-Piquerez',
 'https://fbref.com/en/players/6e5a3bc7/Raphael-Veiga',
 'https://fbref.com/en/players/47431289/Mayke',
 'https://fbref.com/en/players/6ebe6cc1/Luan-Garcia',
 'https://fbref.com/en/players/bd8432ff/Vanderlan',
 'https://fbref.com/en/players/6bc49da7/Gabriel-Veron',
 'https://fbref.com/en/players/01819e70/Gabriel-Menino',
 'https://fbref.com/en/players/434de3a3/Wesley',
 'https://fbref.com/en/players/60990b81/Eduard-Atuesta',
 'https://fbref.com/en/players/e1426a52

<hr>

In [7]:
def fbref_player_info(player_url):
    player_info = []
    for completed, i in enumerate(player_url):
        time.sleep(random.uniform(1,2.9))
      
        print(i)
        # Id
        playerId = i.replace("https://fbref.com/en/players/", "").split("/")[0]

        # Request
        # r=requests.get(i)
        # soup=BeautifulSoup(r.content, "html.parser")
        soup=make_request(i)

        # Meta
        meta = soup.find("div", {"id":"meta"})

        # Nombre
        try:
            playerName = soup.find("h1").find("span").get_text()
        except:
            playerName = 'No Name'
        
        # Foto
        try:
            photo = soup.find("div", {"class":"media-item"}).find("img").attrs["src"]
        except:
            photo = np.nan

        # Data Frame
        temp = pd.DataFrame({
            "FBRefId":[playerId],
            "PlayerName":[playerName],
            "Photo":[photo],
            "PlayerUrl":[i]
        })    

        player_info.append(temp)

        # Print mensaje
        sys.stdout.write("\r{0} Cantidad de jugadores scraped desde FBRef".format(completed+1))
        sys.stdout.flush()

        # Esperamos un segundo
        time.sleep(0.01) 

    # Guardamos la informacion del jugador
    player_info = pd.concat(player_info)
    
    return player_info

# Test
#player_info = fbref_player_info(['https://fbref.com/en/players/9413b19f/Alex-Meret'])
player_info = fbref_player_info(player_url)

# using now() to get current time
# current_time = datetime.now()
#agreguemos una columna más que nos informe cuándo fue la fecha de extracción de estos datos
# player_info['dateExtraction'] = current_time;

player_info.head()

https://fbref.com/en/players/895ef5ec/Dudu
1 Cantidad de jugadores scraped desde FBRefhttps://fbref.com/en/players/81be82e9/Weverton
2 Cantidad de jugadores scraped desde FBRefhttps://fbref.com/en/players/a816dbfb/Danilo
3 Cantidad de jugadores scraped desde FBRefhttps://fbref.com/en/players/10fe831e/Murilo-Cerqueira
4 Cantidad de jugadores scraped desde FBRefhttps://fbref.com/en/players/2964fd20/Gustavo-Scarpa
5 Cantidad de jugadores scraped desde FBRefhttps://fbref.com/en/players/634b1413/Ze-Rafael
6 Cantidad de jugadores scraped desde FBRefhttps://fbref.com/en/players/d5ce52a5/Gustavo-Gomez
7 Cantidad de jugadores scraped desde FBRefhttps://fbref.com/en/players/bffd5e50/Rony
8 Cantidad de jugadores scraped desde FBRefhttps://fbref.com/en/players/fdd2f275/Marcos-Rocha
9 Cantidad de jugadores scraped desde FBRefhttps://fbref.com/en/players/46009f44/Joaquin-Piquerez
10 Cantidad de jugadores scraped desde FBRefhttps://fbref.com/en/players/6e5a3bc7/Raphael-Veiga
11 Cantidad de jugadores 

,FBRefId,PlayerName,Photo,PlayerUrl
0,895ef5ec,Dudu,https://fbref.com/req/202302030/images/headsho...,https://fbref.com/en/players/895ef5ec/Dudu
0,81be82e9,Wéverton,https://fbref.com/req/202302030/images/headsho...,https://fbref.com/en/players/81be82e9/Weverton
0,a816dbfb,Danilo,NaN,https://fbref.com/en/players/a816dbfb/Danilo
0,10fe831e,Murilo Cerqueira,https://fbref.com/req/202302030/images/headsho...,https://fbref.com/en/players/10fe831e/Murilo-C...
0,2964fd20,Gustavo Scarpa,https://fbref.com/req/202302030/images/headsho...,https://fbref.com/en/players/2964fd20/Gustavo-...


In [8]:
player_info

,FBRefId,PlayerName,Photo,PlayerUrl
0,895ef5ec,Dudu,https://fbref.com/req/202302030/images/headsho...,https://fbref.com/en/players/895ef5ec/Dudu
0,81be82e9,Wéverton,https://fbref.com/req/202302030/images/headsho...,https://fbref.com/en/players/81be82e9/Weverton
0,a816dbfb,Danilo,NaN,https://fbref.com/en/players/a816dbfb/Danilo
0,10fe831e,Murilo Cerqueira,https://fbref.com/req/202302030/images/headsho...,https://fbref.com/en/players/10fe831e/Murilo-C...
0,2964fd20,Gustavo Scarpa,https://fbref.com/req/202302030/images/headsho...,https://fbref.com/en/players/2964fd20/Gustavo-...
...,...,...,...,...
0,23f60edd,David Cabrera Pujol,https://fbref.com/req/202302030/images/headsho...,https://fbref.com/en/players/23f60edd/David-Ca...
0,0af95d4b,Jonathan Perlaza,https://fbref.com/req/202302030/images/headsho...,https://fbref.com/en/players/0af95d4b/Jonathan...
0,9e33acc6,Alejandro Arana,NaN,https://fbref.com/en/players/9e33acc6/Alejandr...
0,a982b2e9,Alberto García Jr.,NaN,https://fbref.com/en/players/a982b2e9/Alberto-...


In [9]:
player_info.to_csv('/content/drive/MyDrive/CSV/FBREF/IMG/player_info_'+name_competition_season+'.csv')

<hr>

In [10]:
len(player_info)

1967

In [11]:
def saveIMG(row):

  try:
    path_to_image=row['Photo']
    name=row['PlayerName']
    extension = ''
    if path_to_image.endswith('.jpg') or path_to_image.endswith('.jpeg'):
        mime_type = 'image/jpeg'
        extension = '.jpeg'
    elif path_to_image.endswith('.png'):
        mime_type = 'image/png'
        extension = '.png'
    elif path_to_image.endswith('.gif'):
        mime_type = 'image/gif'
        extension = '.gif'
    else:
        extension = '.png'

    img = Image.open(requests.get(path_to_image, stream=True).raw)
    img = img.resize((250,250), Image.ANTIALIAS)
    img.save("/content/drive/MyDrive/CSV/FBREF/IMG/file/" + name + extension) # Image saving to another directory
  except:
    pass
  return

contador=0
print("\r{0} total de registros. ".format(len(player_info)))

for indice_fila, fila in player_info.iterrows():
  #if fila['FBRefId']=='3bac82ad':

  saveIMG(fila)

  # Mensaje en pantalla: ¿cuántos equipos hemos scrapeado?
  contador=contador+1
  sys.stdout.write("\r{0} registros insertados. ".format(contador))
  sys.stdout.flush()

1967 total de registros. 
1967 registros insertados. 